In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing


In [2]:
data = pd.read_excel('/content/drive/My Drive/Master thesis/DataSet-forUsing/4-hate_speech_mlma_ar_dataset.xlsx')
print("Sample from dataset",data.info())
data = data[['tweet','sentiment']]



# data = data[data.tweet != '#NAME?']
# print("data_is_null \n",data.isnull().sum())

# print("Sample from dataset [Tweets,Sentiment]",data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3353 entries, 0 to 3352
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           3353 non-null   int64 
 1   tweet                3345 non-null   object
 2   sentiment            3353 non-null   object
 3   directness           3353 non-null   object
 4   annotator_sentiment  3353 non-null   object
 5   target               3353 non-null   object
 6   group                3353 non-null   object
dtypes: int64(1), object(6)
memory usage: 183.5+ KB
Sample from dataset None


In [3]:
# Drop rows that contain empty tweet
print("data_is_null \n",data.isnull().sum())
data=data[data.tweet.str.contains('#NAME?',na=False)]
data.info()

data_is_null 
 tweet        8
sentiment    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet      0 non-null      object
 1   sentiment  0 non-null      object
dtypes: object(2)
memory usage: 0.0+ bytes


In [4]:
data = pd.read_excel('/content/drive/My Drive/Master thesis/DataSet-forUsing/4-hate_speech_mlma_ar_dataset.xlsx',usecols=['tweet', 'sentiment'])

data["HS_NOT"] = data.apply(lambda x: "Not-HS" if x["sentiment"] in ['normal'] else "HS",axis =1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3353 entries, 0 to 3352
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet      3345 non-null   object
 1   sentiment  3353 non-null   object
 2   HS_NOT     3353 non-null   object
dtypes: object(3)
memory usage: 78.7+ KB


In [5]:
texts = pd.concat([data['tweet']],axis=0).astype("str")   
# from tensorflow.keras.preprocessing.text import Tokenize
from keras.preprocessing.text import Tokenizer
print(texts)
title_tokeniser = Tokenizer(num_words=2000,split=' ')
title_tokeniser.fit_on_texts(texts)
X = title_tokeniser.texts_to_sequences(texts)
X[:5]


0       صلاة الفجر خير لك من ترديد بول البعير وسبي الن...
1       صراحة نفسي اشوف ولاد الوسخة اللي قالوا مدرب اج...
2       طيب! هي متبرجة وعبايتها ملونه وطالعة من بيتهم ...
3       @user @user انا اوافقك بخصوص السوريين و العراق...
4       هذه السعودية التي شعبها شعب الخيم و بول البعير...
                              ...                        
3348    @user يمشي بخطا ادارتها قيد من الأمريكان ونهب ...
3349    @user مهما حصل هندوس عليهم شويه الرويبضه بس نخ...
3350             الكلب لا يعض اذن اخوه كذابين خنازير @url
3351    @user لأنه صغير ويكتب في قناة خنازير فلن نقرأ ...
3352        الحريم أجمع افلوس وايفون قديم وواتساب ?? ساعة
Name: tweet, Length: 3353, dtype: object


[[1958, 1324, 1325, 90, 3, 4, 5, 1959, 236, 1960, 1961],
 [993, 171, 290, 616, 1326, 24, 617, 1962, 1327, 141, 754, 324, 755, 2],
 [172, 64, 30, 3, 215, 2],
 [1, 1, 34, 756, 757, 10, 618, 142, 756, 325, 65, 1963, 61, 1328, 2],
 [105, 97, 101, 86, 10, 4, 5, 96, 158, 102]]

In [6]:
# pad: to make all input of same length
X = pad_sequences(X)
print(X[:5])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
  1958 1324 1325   90    3    4    5 1959  236 1960 1961]
 [   0    0    0    0    0    0    0    0    0    0    0  993  171  290
   616 1326   24  617 1962 1327  141  754  324  755    2]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0  172   64   30    3  215    2]
 [   0    0    0    0    0    0    0    0    0    0    1    1   34  756
   757   10  618  142  756  325   65 1963   61 1328    2]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0  105   97  101   86   10    4    5   96  158  102]]


In [7]:
embed_dim = 128
max_fatures=2000
lstm_out = 196
import time

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[0]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
start=time.time()
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print("Time to compile model:",time.time()-start)
print(model.summary())


Time to compile model: 0.012439489364624023
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 3353, 128)         256000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 3353, 128)         0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
print((data['HS_NOT']).values)
Y = pd.get_dummies(data['HS_NOT']).values

print(Y)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.15, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

['HS' 'HS' 'HS' ... 'HS' 'HS' 'Not-HS']
[[1 0]
 [1 0]
 [1 0]
 ...
 [1 0]
 [1 0]
 [0 1]]
(2850, 25) (2850, 2)
(503, 25) (503, 2)


In [18]:
from tqdm import tqdm
batch_size = 32
tqdm(model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2))

Epoch 1/7
90/90 - 12s - loss: 0.1843 - accuracy: 0.9246
Epoch 2/7
90/90 - 12s - loss: 0.1536 - accuracy: 0.9375
Epoch 3/7
90/90 - 12s - loss: 0.1409 - accuracy: 0.9421
Epoch 4/7
90/90 - 12s - loss: 0.1224 - accuracy: 0.9523
Epoch 5/7
90/90 - 12s - loss: 0.1176 - accuracy: 0.9516
Epoch 6/7
90/90 - 12s - loss: 0.1218 - accuracy: 0.9481
Epoch 7/7
90/90 - 12s - loss: 0.1004 - accuracy: 0.9575



0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [10]:
twt = ['مسلمي فلسطين بلا سلاح لتبدأ بعدها مجازر اليهود']

#twt = ['الشعب المصري ده اكتر شعب عنصري و وسخ من جوه بطريقه فظيعه يعني إلا من رحم ربي']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = title_tokeniser.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=128,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("hate")
elif (np.argmax(sentiment) == 1):
    print("normal")

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0 594 221 274]]
1/1 - 0s
hate
